In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb

In [2]:
data=pd.read_csv("Data_for_UCI_named.csv")
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [3]:
data.drop("stab", axis=1, inplace=True)

In [4]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
oe=OrdinalEncoder()
data["stabf"]=oe.fit_transform(data[["stabf"]])

In [5]:
X=data.drop("stabf",axis=1)
y=data["stabf"]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=1)

In [7]:
SC=StandardScaler()
X_train=SC.fit_transform(X_train)
X_test=SC.fit_transform(X_test)

In [8]:
#y_train=SC.fit_transform(y_train.reshape(-1, 1)).ravel()

In [9]:
y_train

2694    1.0
5140    1.0
2568    1.0
3671    1.0
7427    1.0
       ... 
2895    0.0
7813    0.0
905     1.0
5192    1.0
235     0.0
Name: stabf, Length: 8000, dtype: float64

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=1)
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=1)

In [11]:
y_pred=rf.predict(X_test)

### Question 14

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.928

In [13]:
lg=lgb.LGBMClassifier(random_state=1)
lg.fit(X_train,y_train)

LGBMClassifier(random_state=1)

### Question 15

In [14]:
lg.score(X_test,y_test)

0.9365

In [15]:
x_g=xgb.XGBClassifier(random_state=1)
x_g.fit(X_train,y_train)

C:\Users\Jordan\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:13:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Question 16

In [16]:
Score=x_g.score(X_test,y_test)
round(Score, 4)

0.946

In [17]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=1)
#Creating hyperparameter grid for ExtraTreesClassifier
rf_grid={'n_estimators': np.arange(10, 1000,50), "max_depth":[None,3,5,10], 
              'max_features': ['auto', 'sqrt', 'log2', None],
              'min_samples_split': np.arange(2,20,2),
              'min_samples_leaf': np.arange(1,20,2)}

In [18]:
model.fit(X_train,y_train)

ExtraTreesClassifier(random_state=1)

In [19]:
model.score(X_test, y_test)

0.926

In [20]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
rs_extratree=RandomizedSearchCV(ExtraTreesClassifier(),param_distributions=rf_grid,cv=5,scoring="accuracy",n_jobs=-1, n_iter=10, verbose=1, random_state=1)
rs_extratree.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   48.5s finished


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([ 10,  60, 110, 160, 210, 260, 310, 360, 410, 460, 510, 560, 610,
       660, 710, 760, 810, 860, 910, 960])},
                   random_state=1, scoring='accuracy', verbose=1)

### Question 17

In [21]:
rs_extratree.best_params_

{'n_estimators': 760,
 'min_samples_split': 6,
 'min_samples_leaf': 3,
 'max_features': 'auto',
 'max_depth': None}

### Question 18

In [22]:
rs_extratree.score(X_test,y_test)

0.928

In [23]:
model = ExtraTreesClassifier(n_estimators=760, min_samples_split=3,min_samples_leaf=3, max_depth=None, random_state=1, max_features= "auto")
model.fit(X_train, y_train)

ExtraTreesClassifier(min_samples_leaf=3, min_samples_split=3, n_estimators=760,
                     random_state=1)

In [24]:
importances = model.feature_importances_

In [25]:
feature_names = list(range(X_train.shape[1]))

In [26]:
indices = np.argsort(importances)[::-1]
for i in range(len(feature_names)):
    print(f"Feature {feature_names[indices[i]]}: {importances[indices[i]]}")

Feature 1: 0.13426878454863883
Feature 0: 0.13173716922834394
Feature 3: 0.12960732034689526
Feature 2: 0.1274173808151859
Feature 10: 0.10247861875823402
Feature 9: 0.09949846164702218
Feature 11: 0.0994083265730466
Feature 8: 0.09135503770980556
Feature 5: 0.022233450381169265
Feature 6: 0.02176711190398954
Feature 7: 0.021736518557479743
Feature 4: 0.01849181953018935


In [27]:
indices = np.argsort(importances)[::-1]
for i in range(len(feature_names)):
    print(f"{feature_names[indices[i]]}: {importances[indices[i]]}")

1: 0.13426878454863883
0: 0.13173716922834394
3: 0.12960732034689526
2: 0.1274173808151859
10: 0.10247861875823402
9: 0.09949846164702218
11: 0.0994083265730466
8: 0.09135503770980556
5: 0.022233450381169265
6: 0.02176711190398954
7: 0.021736518557479743
4: 0.01849181953018935


In [28]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,1.0
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,0.0
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,1.0
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,1.0
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,1.0


We can see here that feature 1 (tau2) has the most important and feature 4 (p1) has the least importance.